In [95]:
%reload_ext autoreload
%autoreload 2

import gym
import newtaxi
import numpy as np
from IPython.display import clear_output
from time import time

In [96]:
kwargs = {'n':10, 'm':10, 'locs':([1,1], [5,2])}
env = gym.make("NewTaxi-v0", **kwargs)

In [97]:
env.render()

*  *  *  *  *  *  *  *  *  *  
*  I  *  *  *  *  *  *  *  *  
*  *  *  *  *  *  *  *  *  *  
*  *  *  *  *  *  *  *  *  X  
*  *  *  *  *  *  *  *  *  *  
*  *  O  *  *  *  *  *  *  *  
*  *  *  *  *  *  *  *  *  *  
*  *  *  *  *  *  *  *  *  *  
*  *  *  *  *  *  *  *  *  *  
*  *  *  *  *  *  *  *  *  *  



In [118]:
out = env.step(action='R')
print(out)
env.render()

({'loc': [5, 2], 'load': 1}, -7, True)
*  *  *  *  *  *  *  *  *  *  
*  I  *  *  *  *  *  *  *  *  
*  *  *  *  *  *  *  *  *  *  
*  *  *  *  *  *  *  *  *  *  
*  *  *  *  *  *  *  *  *  *  
*  *  O  *  *  *  *  *  *  *  
*  *  *  *  *  *  *  *  *  *  
*  *  *  *  *  *  *  *  *  *  
*  *  *  *  *  *  *  *  *  *  
*  *  *  *  *  *  *  *  *  *  



In [119]:
env.state2state_id(env.state)

152

In [120]:
q_table = np.zeros([env.observation_space_n, env.action_space_n])
print(q_table.shape)

(200, 4)


In [127]:
def q_learning(n=100):
    # hyperparameters for q learning
    alpha = 0.5
    gamma = 0.6
    epsilon = 0.1
    
    timelis = []
    action_space = list(env.action_space.keys())
    for i in range(n):

        state = env.reset()
        stateid = env.state2state_id(state)
        epochs = 0
        penalties = 0
        reward = 0
        done = False
        alpha *= 0.9

        start_time = time()
        # done is successful passenger dropoff
        while not done:
            if np.random.uniform(0, 1) < epsilon:
                action = np.random.randint(4)

            else:
                action = np.argmax(q_table[stateid])

            next_state, reward, done = env.step(action_space[action])            
            next_stateid = env.state2state_id(next_state)

            old_value = q_table[stateid, action]

            # we already took action, so we have next state
            next_max = np.max(q_table[next_stateid])
            # update q-value 
            new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
            
            # update q_table
            q_table[stateid, action] = new_value 

            # if making a big mistake, accumulate penalty
            if reward == -10:
                penalties += 1

            # for iteration
            state = next_state
            stateid = env.state2state_id(state)
            
            epochs += 1

            if i % 100 == 0:
                env.render()
                clear_output(wait = True)
                print(f"Episode: {i}")
                
        if i % 100 == 0:        
            end_time = time()
            print(f"Episode: {i}, time: {end_time-start_time}")
            timelis.append(end_time-start_time)

    print("Training finished.\n")
    print(timelis)

In [ ]:
%%time
q_learning(n=10000)

Episode: 7600
*  *  *  *  *  *  *  *  *  *  
*  I  *  *  *  *  *  *  *  *  
*  *  *  *  *  *  *  *  *  *  
*  *  *  *  *  *  *  *  *  *  
X  *  *  *  *  *  *  *  *  *  
*  *  O  *  *  *  *  *  *  *  
*  *  *  *  *  *  *  *  *  *  
*  *  *  *  *  *  *  *  *  *  
*  *  *  *  *  *  *  *  *  *  
*  *  *  *  *  *  *  *  *  *  



In [130]:
q_table

array([[-3299.68242367, -2006.14578827, -3279.28196202,  -880.01592005],
       [-3144.60287285,  -823.42633651, -1926.43714593, -3146.42899546],
       [-3145.07318285, -3215.7255527 , -3185.4184442 , -3807.17434555],
       [-3960.28148547, -3801.38092179, -3910.25469793, -3902.82474112],
       [ -218.20444503, -3888.2167432 , -3762.09674325, -1225.41145745],
       [ -246.77258441, -2692.80250877, -3128.21455255, -2975.18497626],
       [-4035.86440774, -4035.43398718, -1734.64796009, -4026.78032079],
       [-4062.22239322, -3169.27467965, -4048.62096198, -3261.70392536],
       [-4082.13969504, -4093.31609289, -4090.5702342 ,  -584.13158016],
       [ -912.27340902, -3429.55084268, -1034.18271422, -2367.85318931],
       [ -972.77017368, -3063.87394784, -3098.62649851, -2988.22697471],
       [-6088.74999987, -3072.33079094, -6087.9998779 ,  -638.72350186],
       [-3759.95141143, -4008.89679721, -1660.76051378, -3809.2754227 ],
       [-3895.72641762, -3740.94874217, -4118.65351

In [82]:
env.state

{'loc': [5, 2], 'load': 1}